Jusqu'à présent, je me suis concentré sur le fichier train.csv mais il y a aussi authorData qui n'a juste pas l'embedding.

J'avais vu que l'embedding n'était pas méga importante mais je vais chercher à la garder. Pour cela, je vais chercher à estimer lesparamètres de l'embedding.

(Notations : D1 = train.csv; D2 = authorData.csv; N = colonnes non-embedding(=explicites parce que je les connais presque qu'elles sont presques lisibles par un humain); M = colonnes embeddées.)

idée : utiliser train pour apprendre l'embedding sur N colonnes pour en prédire m<=M.
M=1024 donc prédire les 1024 est violent. On se propose de prédire les m termes de la pca

Etapes (d'entrainement et calibration du modèle)
* PCA(m) sur M colonnes de train
* diff train, test et X, Y 
* prédiction(m) à partir de N de X_train
  * modèle
  * métrique

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from notebook_helper import split_url, explicit_part_preparation, pca_transform

In [2]:
df = pd.read_csv("./data/train.csv", sep=",")

### a. PCA

In [10]:
m = 20
df_reduced = pca_transform(df, n_components=m)

In [4]:
df_reduced.head(3)

,engagement,timestamp,language,feature1,feature2,followers,author,word_count,shared_url_count,shared_url_domain,...,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20
0,15,1655784230254,en,73,0,696,bd20432d80dfe4825a7a106312bda52e,18,0,NaN,...,-0.009109,-0.123089,0.036103,0.024297,-0.049868,-0.008181,-0.027987,0.046762,-0.037934,-0.030083
1,91007,1655599613254,en,100,5,17526,71e2ecc6cc1d6515a6b122fab4c63bfc,15,0,NaN,...,-0.098974,0.001679,0.021364,0.001487,-0.059623,-0.012745,-0.000354,0.088753,-0.066704,0.045841
2,3,1655788872254,en,0,-5,239,2a9109f38d90a2a96e284a07c1a57e9b,8,0,NaN,...,-0.026477,-0.009249,0.066340,-0.003064,-0.001726,-0.015418,-0.014679,-0.034675,-0.043635,-0.032478


### b. entraînement

In [5]:
df_prepared = explicit_part_preparation(df_reduced)

In [6]:
df_prepared.head(3)

,engagement,timestamp,feature2,followers,word_count,is_reply,contains_video,contains_image,V1,V2,...,shared_url_extension_net,shared_url_extension_news,shared_url_extension_org,shared_url_extension_other,shared_url_extension_tv,is_shared_url,month_post,day_of_week_post,hour_post,feature1_is_max
0,1.176091,6.727053,0,2.842609,1.278754,False,False,True,0.040117,0.039279,...,False,False,False,True,False,False,6,1,4,False
1,4.959075,8.278642,5,4.243683,1.204120,False,False,False,-0.154545,0.130231,...,False,False,False,True,False,False,6,6,0,True
2,0.477121,5.840107,-5,2.378398,0.954243,True,False,False,0.319861,-0.050030,...,False,False,False,True,False,False,6,1,5,False


In [8]:
V_cols = [col for col in df_prepared.columns if "V" in col]

# Separation of the target
X = df_prepared.drop(columns=V_cols)
y = df_prepared[V_cols]

# Test-train split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [19]:
y_pred_dict = {}

for v_col in V_cols:
    print(v_col)

    # Model training
    model = RandomForestRegressor()
    model.fit(X_train, y_train[v_col])

    # Predict
    y_pred_dict[v_col] = model.predict(X_test)

y_pred = pd.DataFrame(y_pred_list, V_cols).T


V1
V2
V3
V4
V5
V6
V7
V8
V9
V10
V11
V12
V13
V14
V15
V16
V17
V18
V19
V20


In [22]:
for v_col in V_cols:
    mse = mean_squared_error(y_test[v_col], y_pred_dict[v_col])
    print(f"{v_col} mse = {mse}")

V1 mse = 0.013660232010557008
V2 mse = 0.004638162640453503
V3 mse = 0.008012535927999918
V4 mse = 0.002839110161222609
V5 mse = 0.003650305344898822
V6 mse = 0.002559243128368535
V7 mse = 0.0023109861637855586
V8 mse = 0.0017576502402389975
V9 mse = 0.0017916175966169903
V10 mse = 0.0017645829231514802
V11 mse = 0.0018405837342305574
V12 mse = 0.0013727990650523466
V13 mse = 0.00149148852838975
V14 mse = 0.0014085544595553838
V15 mse = 0.0013559852701161247
V16 mse = 0.0014093645364430915
V17 mse = 0.001372672717025847
V18 mse = 0.0012492482758767998
V19 mse = 0.001378949277281156
V20 mse = 0.0012761901688047386


In [ ]:
# Separation of the target
X = df_encoded_reduced.drop(columns=["engagement"])
y = df_encoded_reduced["engagement"]

# Test-train split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

# Model training
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Eval on test split
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R^2 score:", r2)

gs.com
firstname.surname@gd.com
networking events

Camille Humbert
Lore Magnan
Arnaud Sadaca / Sodaca
Niculas Durand